In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore, db
import json

In [4]:
# firebase login stuff

### make a credential json...

cred = credentials.Certificate("C:\\Users\\erussek\\forage_jsp\\analysis\\foragetask-firebase-private-key.json")
default_app = firebase_admin.initialize_app(cred)
client = firestore.client()

In [106]:
dfs = []
bonus_df = pd.DataFrame()
for subj in client.collection('foragetask').document('run1').collection('subjects').stream():
    subj_dfs = []
    taskdata_collection = client.collection('foragetask/run1/subjects/{0}/taskdata'.format(subj.id)).stream()
    has_end = False # did this subject finish the task?
    has_start = False
    for tc in taskdata_collection:
        # first thing is start... 
        if tc.id == "end":
            has_end = True
            end_dict = tc.to_dict()
            end_time = end_dict['end_time']
            bonus_points = end_dict['bonus_points'][0]
        elif tc.id == "start":
            has_start = True
            start_dict = tc.to_dict()
            subjectID = start_dict['subjectID']
            date = start_dict['date']
        else:
            trial_dict = tc.to_dict()
            trial_name = tc.id
            trial_data_json = json.loads(trial_dict['trial_data'])
            trial_df = pd.DataFrame(trial_data_json)
            if has_start:
                trial_df['subjectID'] = subjectID
                trial_df['date'] = date
            else:
                trial_df['subjectID'] = None
                trial_df['date'] = None
            if has_end:
                trial_df['bonus_points'] = bonus_points
            else:
                trial_df['bonus_points'] = None
                
            subj_dfs.append(trial_df)
            
    if (has_start & has_end & (len(str(subjectID)) > 20)):
        bonus_dict = {'subjectID': subjectID, 'bonus_points': bonus_points}
        bonus_df = bonus_df.append(bonus_dict, ignore_index = True)
        dfs = dfs + subj_dfs


combined_df = pd.concat(dfs, sort = True)
combined_df.to_csv('data\\run1_data.csv')

# make a structure w/ bonus points... 
def compute_bonus(row):
    pct = np.min([np.max([row['bonus_points'] / (30000 - 0),0]),1])
    return 2.00*pct
                  
bonus_df['bonus'] = bonus_df.apply(compute_bonus, axis = 1)
bonus_df[['subjectID', 'bonus']].to_csv('bonus_payment_files\\run1_bonus_payments.csv', sep = ',', header = False, index = False)
